# Register files from Census release 2023-11-13

In [1]:
import lamindb as ln
import lnschema_bionty as lb

💡 lamindb instance: theislab/nicheformer


In [2]:
census_version = "2023-11-13"

In [3]:
s3path = f"s3://cellxgene-data-public/cell-census/{census_version}/h5ads"
ln.UPath(s3path).view_tree()

h5ads (0 sub-directories & 1122 files with suffixes '.h5ad'): 
├── 00099d5e-154f-4a7a-aa8d-fa30c8c0c43c.h5ad
├── 0041b9c3-6a49-4bf7-8514-9bc7190067a7.h5ad
├── 00476f9f-ebc1-4b72-b541-32f912ce36ea.h5ad
├── 00e5dedd-b9b7-43be-8c28-b0e5c6414a62.h5ad
├── 00ff600e-6e2e-4d76-846f-0eec4f0ae417.h5ad
├── 01209dce-3575-4bed-b1df-129f57fbc031.h5ad
...


In [4]:
ln.track()

💡 notebook imports: lamindb==0.61.0 lnschema_bionty==0.34.0 requests==2.31.0
💡 loaded: Transform(uid='4hbVFAINneXCz8', name='Register files from Census release 2023-11-13', short_name='census-release-2023-11', version='0', type='notebook', updated_at=2023-11-20 21:25:20 UTC, created_by_id=1)
💡 loaded: Run(uid='6ZcVntFiofyv226ywRDL', run_at=2023-11-21 20:58:33 UTC, transform_id=18, created_by_id=1)


## Register files

In [5]:
files = ln.File.from_dir(s3path)
ln.save(files)

✅ created 1122 files from directory using storage s3://cellxgene-data-public and key = cell-census/2023-11-13/h5ads/


In [6]:
dataset = ln.Dataset(files, name="cellxgene-census", version=census_version)
dataset.save()

💡 initializing versioning for this dataset! create future versions of it using ln.Dataset(..., is_new_version_of=old_dataset)


## Register metadata

Get all datasets and associated metadata using cellxgene REST API:

In [5]:
import requests


def get_datasets_df_from_cxg():
    api_url_base = "https://api.cellxgene.cziscience.com"
    datasets_path = "/curation/v1/datasets"
    datasets_url = f"{api_url_base}{datasets_path}"
    headers = {"Content-Type": "application/json"}
    res = requests.get(url=datasets_url, headers=headers)
    res.raise_for_status()
    res_content = res.json()
    return res_content

In [6]:
res_content = get_datasets_df_from_cxg()
len(res_content)

1130

In [7]:
res_content[0].keys()

dict_keys(['assay', 'assets', 'cell_count', 'cell_type', 'collection_doi', 'collection_id', 'collection_name', 'collection_version_id', 'dataset_id', 'dataset_version_id', 'development_stage', 'disease', 'donor_id', 'explorer_url', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'primary_cell_count', 'processing_status', 'published_at', 'revised_at', 'schema_version', 'self_reported_ethnicity', 'sex', 'suspension_type', 'tissue', 'title', 'tombstone', 'x_approximate_distribution'])

In [8]:
features = ln.Feature.lookup()
files = ln.File.filter(key__contains=census_version).all()

### collections, organisms

In [20]:
is_collection = ln.ULabel.filter(name="is_collection").one()
collections = is_collection.children.all()
organisms = lb.Organism.filter().all()

for dataset_meta in res_content:
    file = files.filter(key__contains=dataset_meta["dataset_id"]).one_or_none()
    if file is None:
        continue
    # register collection
    collection = collections.filter(
        reference=dataset_meta["collection_id"]
    ).one_or_none()
    if collection is None:
        collection = ln.ULabel(
            name=dataset_meta["collection_name"],
            description=dataset_meta["collection_doi"],
            reference=dataset_meta["collection_id"],
            reference_type="collection_id",
        )
        collection.save()
        collection.parents.add(is_collection)
    file.labels.add(collection, feature=features.collection)

    # annotate with organism
    organism_ontology_ids = [i["ontology_term_id"] for i in dataset_meta["organism"]]
    organism_records = organisms.filter(ontology_id__in=organism_ontology_ids).list()
    # register new organisms
    if len(organism_records) == 0:
        bionty_source = lb.BiontySource.filter(entity="Organism", organism="all").one()
        for i in organism_ontology_ids:
            record = lb.Organism.from_bionty(ontology_id=i, bionty_source=bionty_source)
            record.save(parents=False)
            print(f"registered organism: {record}")
        organism_records = organisms.filter(
            ontology_id__in=organism_ontology_ids
        ).list()
    file.labels.add(organism_records, feature=features.organism)

❗ loading non-default source inside a LaminDB instance
registered organism: Organism(uid='sMq3BQz0', name='rhesus macaque', ontology_id='NCBITaxon:9544', scientific_name='Macaca mulatta', updated_at=2023-11-21 12:30:09 UTC, bionty_source_id=50, created_by_id=1)
❗ loading non-default source inside a LaminDB instance
registered organism: Organism(uid='aYpsHRNc', name='white-tufted-ear marmoset', ontology_id='NCBITaxon:9483', scientific_name='Callithrix jacchus', updated_at=2023-11-21 12:30:18 UTC, bionty_source_id=50, created_by_id=1)
❗ loading non-default source inside a LaminDB instance
registered organism: Organism(uid='Ihbc7tQx', name='chimpanzee', ontology_id='NCBITaxon:9598', scientific_name='Pan troglodytes', updated_at=2023-11-21 12:30:26 UTC, bionty_source_id=50, created_by_id=1)
❗ loading non-default source inside a LaminDB instance
registered organism: Organism(uid='Z0wGZNxo', name='domestic pig', ontology_id='NCBITaxon:9825', scientific_name='Sus scrofa domesticus', updated_a

### obs ontologies

In [9]:
feature_names = [
    "self_reported_ethnicity",
    "development_stage",
    "cell_type",
    "assay",
    "tissue",
    "disease",
    "sex",
    "donor_id",
    "suspension_type",
]

from lamindb.dev._feature_manager import get_accessor_by_orm

ACCESSORS = get_accessor_by_orm(ln.File)
FEATURE_TO_ACCESSOR = {}
for name in feature_names:
    feature = getattr(features, name)
    accessor = ACCESSORS.get(feature.registries)
    orm = getattr(ln.File, accessor).field.model
    # TODO: ulabels are defined in the File model, improve this in LaminDB
    if orm == ln.File:
        orm = getattr(ln.File, accessor).field.related_model
    FEATURE_TO_ACCESSOR[name] = (accessor, orm)

In [37]:
obs_featureset = ln.FeatureSet(features=[getattr(features, i) for i in feature_names])
obs_featureset.save()

obs_featureset.files.set(files, through_defaults={"slot": "obs"})

In [10]:
# extra step to register uberon ontologies as developmental stages
def create_dv_record_from_uberon(ontology_id: str):
    tissue_record = lb.Tissue.from_bionty(ontology_id=ontology_id)
    dvs_record = lb.DevelopmentalStage(
        name=tissue_record.name,
        description=tissue_record.description,
        ontology_id=tissue_record.ontology_id,
        bionty_source_id=tissue_record.bionty_source_id,
    )
    dvs_record.save()


for id in [
    "UBERON:0018241",
    "UBERON:0034919",
    "UBERON:0007222",
    "UBERON:0000113",
    "UBERON:0007220",
    "UBERON:0007222",
]:
    create_dv_record_from_uberon(id)

In [10]:
ontology_ids = {}
for name in feature_names:
    if name in ["donor_id", "suspension_type"]:
        continue
    allids = set()
    for i in res_content:
        if name in i:
            allids.update([(j["label"], j["ontology_term_id"]) for j in i[name]])

    ontology_ids[name] = allids

# register all ontology ids
for name, terms in ontology_ids.items():
    accessor, orm = FEATURE_TO_ACCESSOR.get(name)
    terms_ids = [i[1] for i in terms]
    records = orm.from_values(terms_ids, field="ontology_id")
    if len(records) > 0:
        ln.save(records)

❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ did not create DevelopmentalStage records for 19 non-validated ontology_ids: 'MmusDv:0000057', 'MmusDv:0000058', 'MmusDv:0000059', 'MmusDv:0000065', 'MmusDv:0000066', 'MmusDv:0000067', 'MmusDv:0000068', 'MmusDv:0000070', 'MmusDv:0000071', 'MmusDv:0000072', 'MmusDv:0000073', 'MmusDv:0000074', 'MmusDv:0000079', 'MmusDv:0000098', 'MmusDv:0000099', 'MmusDv:0000102', 'UBERON:0000113', 'UBERON:0007220', 'UBERON:0007222'
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ did not create Tissue records for 18 non-validated ontology_ids: 'CL:0000010 (cell culture)', 'CL:0000082 (cell culture)', 'CL:0000084 (cell culture)', 'CL:0000115 (cell culture)', 'CL:0000351 (cell culture)', 'CL:0002322 (cell culture)', 'CL:0002327 (cell culture)', 'CL:0002328 (cell culture)', 'C

In [16]:
# register the non-validated terms
bionty_source_ds_mouse = lb.BiontySource.filter(
    entity="DevelopmentalStage", organism="mouse"
).one()

for name, terms in ontology_ids.items():
    accessor, orm = FEATURE_TO_ACCESSOR.get(name)
    terms_ids = [i[1] for i in terms]
    result = orm.inspect(terms_ids, field="ontology_id")
    if len(result.non_validated) > 0:
        if name == "development_stage":
            dv_records = orm.from_values(
                result.non_validated,
                field="ontology_id",
                bionty_source=bionty_source_ds_mouse,
            )
        elif name == "tissue":
            ts_records = [
                orm(name=term[0], ontology_id=term[1])
                for term in terms
                if term[1] in result.non_validated
            ]

ln.save(dv_records)
ln.save(ts_records)

❗ 16 terms (7.00%) are not validated for ontology_id: MmusDv:0000066, MmusDv:0000057, MmusDv:0000072, MmusDv:0000067, MmusDv:0000099, MmusDv:0000070, MmusDv:0000065, MmusDv:0000071, MmusDv:0000058, MmusDv:0000102, MmusDv:0000098, MmusDv:0000068, MmusDv:0000079, MmusDv:0000059, MmusDv:0000073, MmusDv:0000074
   couldn't validate 16 terms: 'MmusDv:0000059', 'MmusDv:0000068', 'MmusDv:0000058', 'MmusDv:0000066', 'MmusDv:0000057', 'MmusDv:0000102', 'MmusDv:0000098', 'MmusDv:0000071', 'MmusDv:0000074', 'MmusDv:0000065', 'MmusDv:0000067', 'MmusDv:0000073', 'MmusDv:0000070', 'MmusDv:0000072', 'MmusDv:0000079', 'MmusDv:0000099'
→  if you are sure, create new records via ln.DevelopmentalStage() and save to your registry
❗ 18 terms (4.80%) are not validated for ontology_id: UBERON:0001295 (organoid), UBERON:0002370 (organoid), CL:0002327 (cell culture), CL:0000115 (cell culture), CL:0000082 (cell culture), CL:0002334 (cell culture), CL:0002335 (cell culture), CL:0000010 (cell culture), CL:0002328

### donors and suspension types

In [31]:
donor_ids = set()
suspension_types = set()

for i in res_content:
    if "donor_id" in i:
        donor_ids.update(i["donor_id"])
    if "suspension_type" in i:
        suspension_types.update(i["suspension_type"])

In [18]:
donors = ln.ULabel.filter(name="is_donor").one().children.all()
result = donors.inspect(donor_ids)
new_donors = [ln.ULabel(name=name) for name in result.non_validated]
ln.save(new_donors)
is_donor = ln.ULabel.filter(name="is_donor").one()
is_donor.children.add(*new_donors)

❗ 1216 terms (16.60%) are not validated for name: mouse_YUYDJ, COVID19_Participant12, COVID19_Participant3, 328806, 364368, Human-BRCA1-B, RL2105, COVID19_Participant6, 1864, mouse_LVGJZ, Control_Participant14, 424707, 291740, 377001, mouse_VBJMY, G19.32.002, mouse_EUEJJ, mouse_RZFCS, mouse003, HTA4_11, ...
   couldn't validate 1216 terms: 'Control_Participant16', 'C19.32.004', 'mouse_YUYDJ', '14104', 'Williams_GSE164241_GSM5177040', '5976', 'H18.29.134', 'COVID19_Participant12', 'mouse_VTQGY', 'ac45', 'HTA11_99999974143_84620', '779', 'AH2', 'Kydar05', 'GW20-12-7-18', 'Human-WT-C', 'Q19.26.003', 'SPECTRUM-OV-031', '380739', '514', ...
→  if you are sure, create new records via ln.ULabel() and save to your registry


In [36]:
stypes = ln.ULabel.filter(name="is_suspension_type").one().children.all()
result = stypes.inspect(suspension_types)
new_stypes = [ln.ULabel(name=name) for name in result.non_validated]
ln.save(new_stypes)
is_suspension_type = ln.ULabel.filter(name="is_suspension_type").one()
is_suspension_type.children.add(*new_stypes)

❗ 1 term (33.30%) is not validated for name: na
   couldn't validate 1 term: 'na'
→  if you are sure, create new record via ln.ULabel() and save to your registry
❗ records with similar names exist! did you mean to load one of them?


,uid,score
name,,
nan,etXMfuAG,90.0
mouse_NASMM,H1NNCpCh,90.0


## Annotate files

In [45]:
for idx, dataset_meta in enumerate(res_content):
    if idx % 50 == 0:
        print(f"annotating dataset {idx} of {len(res_content)}")
    file = files.filter(key__contains=dataset_meta["dataset_id"]).one_or_none()
    if file is None:
        continue
    for field, terms in dataset_meta.items():
        if field not in FEATURE_TO_ACCESSOR:
            continue
        accessor, orm = FEATURE_TO_ACCESSOR.get(field)
        if field in ["donor_id", "suspension_type"]:
            records = orm.from_values(terms, field="name")
        else:
            records = orm.from_values(
                [i["ontology_term_id"] for i in terms], field="ontology_id"
            )
        if len(records) > 0:
            getattr(file, accessor).add(*records)

annotating dataset 0 of 1130
annotating dataset 50 of 1130
annotating dataset 100 of 1130
annotating dataset 150 of 1130
annotating dataset 200 of 1130
annotating dataset 250 of 1130
annotating dataset 300 of 1130
annotating dataset 350 of 1130
annotating dataset 400 of 1130
annotating dataset 450 of 1130
annotating dataset 500 of 1130
annotating dataset 550 of 1130
annotating dataset 600 of 1130
annotating dataset 650 of 1130
annotating dataset 700 of 1130
annotating dataset 750 of 1130
annotating dataset 800 of 1130
annotating dataset 850 of 1130
annotating dataset 900 of 1130
annotating dataset 950 of 1130
annotating dataset 1000 of 1130
annotating dataset 1050 of 1130
annotating dataset 1100 of 1130


In [46]:
files.last().describe()

File(uid='aEKsguSJF6kAoeXcICKt', key='cell-census/2023-11-13/h5ads/ff7d15fa-f4b6-4a0e-992e-fd0c9d088ded.h5ad', suffix='.h5ad', accessor='AnnData', size=339098252, hash='wk4aVyHI7iZWNq2n99_s4w-41', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-11-20 21:45:35 UTC)

Provenance:
  🗃️ storage: Storage(uid='oIYGbD74', root='s3://cellxgene-data-public', type='s3', region='us-west-2', updated_at=2023-10-16 15:04:08 UTC, created_by_id=1)
  📔 transform: Transform(uid='4hbVFAINneXCz8', name='Register files from Census release 2023-11-13', short_name='census-release-2023-11', version='0', type='notebook', updated_at=2023-11-20 21:25:20 UTC, created_by_id=1)
  👣 run: Run(uid='6ZcVntFiofyv226ywRDL', run_at=2023-11-21 19:43:44 UTC, transform_id=18, created_by_id=1)
  👤 created_by: User(uid='kmvZDIX9', handle='sunnyosun', name='Sunny Sun', updated_at=2023-11-21 09:23:50 UTC)
Features:
  external: FeatureSet(uid='qgAXuV6LwRitejVy0ZY9', n=2, registry='core.Feature', hash='NspE1Q

## Register genes

In [ ]:
ln.settings.track_run_inputs = False
ln.settings.verbosity = "hint"

for file in files:
    adata_backed = file.backed()
    genes = adata_backed.var_names
    featureset = ln.FeatureSet.from_values(
        genes, field=lb.Gene.ensembl_gene_id, organism=file.organism.first()
    )
    featureset.save()
    file.feature_sets.add(featureset, through_defaults={"slot": "var"})